In [1]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

In [4]:
# Load simulated data
np.random.seed(11)
parlist, xslist, yslist = [], [], []

# Training set should be one concatenated shuffled list of simulated examples
# Validation set similarly, but also keep information about simulation parameters
# to evaluate performance in specific simulation conditions.
for bounds_file in glob.glob("../data/simulData/*_bmBounds.tab"):
    m = re.search(r"/scale(.*)-adip(.*)_bmBounds.tab$", bounds_file)
    scale, adip = [float(x) for x in m.groups()]
    parlist.append((scale, adip))
    ys = np.loadtxt(bounds_file)
    intensities_file = re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
    xs = np.loadtxt(intensities_file)
    #yslist.append(ys)
    #xslist.append(xs)
    #xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 11)

In [5]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='relu')
    ])
model.compile(optimizer=tf.train.AdamOptimizer(), loss='mse', metrics=['mae'])

In [6]:
model.fit(x, y, epochs=10, batch_size=32, validation_split=0.33)

NameError: name 'x' is not defined